In [ ]:
import seaborn
import pandas as pd
import numpy as np


In [ ]:
titanic = seaborn.load_dataset('Titanic')
titanic.sample()

# Task 1

In [ ]:
# Split the target 'survived' off from the predictors

In [ ]:
# convert the 'sex' column to a binary where 1 represents female

# Task 2

In [ ]:
# Implement a train-test split using all default arguments and random_state=42

# Task 3

We will fit a logistic regression model using only `sex` and `fare`.  Remember that logistic regression uses regularization by default.  That means that scaling will have an effect on the scores. 

In [ ]:
# Fit a standard scaler to train columns of interest, and transform the test
ss = None
X_train_sc = None
X_test_sc = None

In [ ]:
# Instantiate a logistic regression model with the default arguments and random_state=42
lr = None
# Fit on the scaled training data. 


# Task 4

The coefficients produced by fitting the model dictate the predictions that the model makes.

In [ ]:
# Model test predictions
y_hat = lr.predict(X_test_sc)

Under the hood, the model is applying a decision threshold to assign a 0 or 1 prediction. 
For the exercise below, apply a decision threshold of .5 to the predicted probabities, so that you can recreate 0 or 1 predictions stored in y_hat above.

In [ ]:
# Convert these predicted probabilities to 0/1 predictions
y_hat_proba = lr.predict_proba(X_test_sc)

# Apply threshold here
y_hat_by_hand = None

In [ ]:
assert (y_hat == y_hat_by_hand).sum() == len(y_hat)

# Stretch Task

For the final task, we will recreate the predicted predicted probabilities using the coefficients.

In [ ]:
def sigmoid(log_odd):
    
    '''
    The link function translates a log_odd prediction
    and returns a probability of class 1 with a
    number between 0 and 1.
    '''
    
    return 1/(1+np.e**(-log_odd))
    

In the cell below, use the coef_ and intercept_ attributes from the fit model to calculate the log_odds for each record in the test set. Then pass these log_odds into the sigmoid function above.

In [ ]:
# Log odds are the output of the dot product of X_test, 
# a version of the coef_ attribute, plus the interceps_ attribute. 
log_odds = None

# use a list comprehension to apply the sigmoid function to each log_odd.
predict_proba_by_hand = None

In [ ]:
assert np.isclose(predict_proba_by_hand, y_hat_proba[:,1]).sum() == 223